In [48]:
from hashlib import sha256
import json
import time


#block

class block:
    def __init__(self, index, transaction, timestample, prev_hash):
        self.index = index
        self.transaction = transaction
        self.timestample = timestample
        self.prev_hash = prev_hash
        
    def compute_hash(self):
    
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()

class transaction:
    def __init__(self, SN, from_key, to_key, validation):
        self.SN = SN
        self.from_key = from_key
        self.to_key = to_key
        self.validation = validation    


chain = dict()
chain['b0'] = block(0,transaction('sis',"gene","n",True),0,0)



def send(from_key, to_key, SN):                   #mandar a transação primeiro para o bloco depois para a chain
    if SN in from_key.parts:
        from_key.parts.pop(from_key.parts.index(SN))
        to_key.parts.append(SN)
        print('part sent')
        
        key = 'b{}'.format(len(chain))
        chain[key] = block(len(chain),transaction(SN, from_key, to_key, True),'time',hash(str(chain['b{}'.format(len(chain)-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)-1)].transaction.SN)))        
        #para recuperar os dados cache["ab123"].SN/from_key...
        
    
    elif SN not in from_key.parts:
        print('Part not in your wallet')
    
    
class wallet:
    def __init__(self,parts,public_key,private_key):
        self.parts = parts
        self.public_key = public_key
        self.private_key = private_key

    import random
    import string
    
    def new_wallet(username):    # 
        username = wallet([],(''.join(random.choices(string.ascii_uppercase + string.digits, k=8))),123)
        
    
def check_chain():
    last_block = -1
    previous_hash = hash(str(chain['b{}'.format(len(chain)+last_block-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)+last_block-1)].transaction.SN))
        
    
    while len(chain) + prev_b +1 != 0:
        if chain['b{}'.format(len(chain)+last_block)].prev_hash == previous_hash:
            print ('pass')
            prev_b = prev_b - 1
            last_block = last_block - 1         #está marcando todos os blocos como inválidos
        else:
            print ('Block {} invalid'.format(len(chain)+last_block))
            prev_b = prev_b - 1
            last_block = last_block - 1
        
        
        
        #criar sistema de explorer
        #ex: hist_part(SN)


In [40]:
last_block = -1
hash(str(chain['b{}'.format(len(chain)+last_block-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)+last_block-1)].transaction.SN))

-5332996661355439259

In [42]:
chain['b{}'.format(len(chain)+last_block)].prev_hash

-5332996661355439259

In [47]:
check_chain()

pass
Block 2 invalid
Block 1 invalid


In [44]:
def check_chain():
    prev_b = -2
    last_block = -1
    previous_hash = hash(str(chain['b{}'.format(len(chain)+last_block-1)].transaction.from_key)+str(chain['b{}'.format(len(chain)+last_block-1)].transaction.SN))
    
        
    
    while len(chain) + prev_b +1 != 0:
        if chain['b{}'.format(len(chain)+last_block)].prev_hash == previous_hash:
            print ('pass')
            prev_b = prev_b - 1
            last_block = last_block - 1
        else:
            print ('Block {} invalid'.format(len(chain)+last_block))
            prev_b = prev_b - 1
            last_block = last_block - 1

In [28]:
len(chain)

4

In [45]:
chain

{'b0': <__main__.block at 0x7f1ca45de668>}

In [24]:
x = 2
hash(str(chain['b{}'.format(x)].transaction.from_key)+str(chain['b{}'.format(x)].transaction.SN))


-4772341646734428181

In [46]:
a = wallet([123],'a','a1')
b = wallet([],'b','b1')
c = wallet([],'c','c1')
d = wallet([],'d','d1')
send(a,b,123)
chain

send(b,c,123)
chain

send(c,d,123)
chain

part sent
part sent
part sent


{'b0': <__main__.block at 0x7f1ca45de668>,
 'b1': <__main__.block at 0x7f1ca463d198>,
 'b2': <__main__.block at 0x7f1ca463dc88>,
 'b3': <__main__.block at 0x7f1ca463dbe0>}

In [16]:
chain

{'b0': <__main__.block at 0x7f1ca4657198>,
 'b1': <__main__.block at 0x7f1ca46b9588>,
 'b2': <__main__.block at 0x7f1ca46b9780>}

In [16]:
hash((chain['b{}'.format(len(chain)-1)].transaction.from_key)+(chain['b{}'.format(len(chain)-1)].transaction.SN))

-8493687677552911038

In [26]:
chain['b1'].prev_hash = 2

In [11]:
chain['b1'].prev_hash

1224576515108961723